In [1]:
%load_ext autoreload
%autoreload 1
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

In [2]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
import timm
from timm import optim, scheduler
import torch
from torchvision import transforms as tfm
from sklearn import metrics as skmet
import matplotlib.pyplot as plt
import json
import transforms as my_transforms

%aimport dataset
ImageData = dataset.ImageData

In [3]:
# artifact_folder = '/zfs/wficai/pda/model_run_artifacts/20220818_justcolor_224x224'
artifact_folder = '/zfs/wficai/pda/model_run_artifacts/20220818_no2d_224x224'

with open(artifact_folder + '/config.json', 'r') as f: 
    cfg = json.load(f)

# put all config variables in scope to avoid the need to laboriously index cfg
for k, v in cfg.items():
    v = f"'{v}'" if type(v)==str else v
    exec(f"{k}={v}")
del cfg

In [4]:
# optionally override settings
view_filter = ['pdaView', 'pdaRelatedView']
mode_filter = ['color', 'color_compare']
device = torch.device('cuda:1')  # you may need 'cuda:0'

In [5]:
tfms = my_transforms.ImageTransforms(res)
tfms_test = tfms.get_transforms(transforms['test'])

In [6]:
df_test = pd.read_csv(f'{artifact_folder}/{out_paths["test"]}')
d_test = ImageData(df_test, transforms = tfms_test, mode_filter = mode_filter, view_filter = view_filter)
dl_test = DataLoader(d_test, batch_size=bs_test, num_workers=num_workers)

print("Number of frames after filtering:", len(d_test.data))

Number of frames after filtering: 10949


In [7]:
# create model
m = timm.create_model(model, pretrained=pretrained, checkpoint_path = f"{artifact_folder}/model_checkpoint.ckpt", num_classes=num_classes, in_chans=3)
m.to(device)           
m.eval()

loss_function = torch.functional.F.binary_cross_entropy_with_logits

target_ls = []
output_ls = []
study_ls = []
video_ls = []
view_ls = []
mode_ls = []
losses = []

for ix, batch in enumerate(dl_test):
    print(f"Batch {ix+1}", end = "\r")
    inputs = batch['img'].to(device)
    targets = batch['trg_type'].to(device).type(torch.float32)
    target_ls.append(targets.cpu().numpy())
    view_ls.append(batch['trg_view'].numpy())
    mode_ls.append(batch['trg_mode'].numpy())
    study_ls += batch['study']
    video_ls += batch['video']

    with torch.no_grad():
        outputs = m(inputs)
        output_ls.append(outputs.cpu().numpy())
        loss = loss_function(outputs.squeeze(), targets)
        losses.append(loss.detach().item())

In [8]:
df_results = pd.DataFrame(dict(
    study = study_ls,
    video = video_ls,
    predicted = np.concatenate(output_ls).squeeze(),
    target = np.concatenate(target_ls), 
    mode = np.concatenate(mode_ls),
    view = np.concatenate(view_ls)
))

df_results['mode'] = df_results['mode'].map(ImageData.inv_mode_map)
df_results.view = df_results.view.map(ImageData.inv_view_map)

df_results.predicted = 1 / (1 + np.exp(-df_results.predicted))

df_results.head(20)

,study,video,predicted,target,mode,view
0,study11,study11_dicom36,0.024386,0.0,color_compare,pdaView
1,study11,study11_dicom36,0.070581,0.0,color_compare,pdaView
2,study11,study11_dicom36,0.158714,0.0,color_compare,pdaView
3,study11,study11_dicom36,0.060949,0.0,color_compare,pdaView
4,study11,study11_dicom36,0.215609,0.0,color_compare,pdaView
5,study11,study11_dicom36,0.040832,0.0,color_compare,pdaView
6,study11,study11_dicom36,0.111885,0.0,color_compare,pdaView
7,study11,study11_dicom36,0.399966,0.0,color_compare,pdaView
8,study11,study11_dicom36,0.044603,0.0,color_compare,pdaView
9,study11,study11_dicom36,0.119814,0.0,color_compare,pdaView


In [9]:
def compute_metrics(y_true, y_pred, thresh=0.5):
    mets = dict()
    
    y_pred_cls = (y_pred>thresh).astype(int)
    
    mets['num_samples'] = len(y_true)
    mets['roc_auc'] = skmet.roc_auc_score(y_true, y_pred)
    mets['average_precision'] = skmet.average_precision_score(y_true, y_pred)
    mets['accuracy'] = skmet.accuracy_score(y_true, y_pred_cls)
    mets['sensitivity'] = skmet.recall_score(y_true, y_pred_cls)
    mets['specificity'] = skmet.recall_score(y_true, y_pred_cls, pos_label=0)
    
    return mets

# Frame-level results

In [10]:
compute_metrics(df_results.target, df_results.predicted)

{'num_samples': 10949,
 'roc_auc': 0.8156664667185259,
 'average_precision': 0.7630163911913119,
 'accuracy': 0.7239930587268244,
 'sensitivity': 0.772520908004779,
 'specificity': 0.6939680662329982}

In [11]:
grouped_results = df_results.groupby(['view', 'mode']).apply(lambda dat: compute_metrics(dat.target, dat.predicted))
grouped_results = pd.DataFrame(grouped_results.tolist(), index=grouped_results.index)
grouped_results

num_samples   roc_auc  average_precision  \
view           mode                                                      
pdaRelatedView color                 2756  0.885659           0.672251   
               color_compare         3094  0.652287           0.585206   
pdaView        color                 1680  0.967213           0.958566   
               color_compare         3419  0.842250           0.842462   

                              accuracy  sensitivity  specificity  
view           mode                                               
pdaRelatedView color          0.771771     0.872047     0.749110  
               color_compare  0.594376     0.592218     0.595909  
pdaView        color          0.841667     0.950078     0.774783  
               color_compare  0.744955     0.810965     0.675659

# Clip-level results

### Avg confidence over frames

In [12]:
df_results_clip_avg = df_results.groupby(['study', 'video', 'target', 'view', 'mode'], as_index=False).agg('mean')
display(df_results_clip_avg.head(10))

compute_metrics(df_results_clip_avg.target, df_results_clip_avg.predicted)

,study,video,target,view,mode,predicted
0,study1,study1_dicom11,1.0,pdaView,color,0.937958
1,study1,study1_dicom12,1.0,pdaView,color,0.975192
2,study1,study1_dicom15,1.0,pdaView,color,0.973028
3,study1,study1_dicom17,1.0,pdaRelatedView,color,0.895997
4,study1,study1_dicom29,1.0,pdaRelatedView,color,0.761491
5,study1,study1_dicom46,1.0,pdaRelatedView,color,0.759022
6,study11,study11_dicom36,0.0,pdaView,color_compare,0.131625
7,study11,study11_dicom49,0.0,pdaView,color_compare,0.380988
8,study12,study12_dicom19,0.0,pdaRelatedView,color,0.347568
9,study12,study12_dicom25,0.0,pdaRelatedView,color,0.543069


{'num_samples': 123,
 'roc_auc': 0.8952013237727523,
 'average_precision': 0.8756761138488989,
 'accuracy': 0.8048780487804879,
 'sensitivity': 0.8367346938775511,
 'specificity': 0.7837837837837838}

In [13]:
grouped_results_clip_avg = df_results_clip_avg.\
    groupby(['view', 'mode']).\
    apply(lambda dat: compute_metrics(dat.target, dat.predicted, thresh=0.55))
grouped_results_clip_avg = pd.DataFrame(grouped_results_clip_avg.tolist(), index=grouped_results.index)
grouped_results_clip_avg

num_samples   roc_auc  average_precision  \
view           mode                                                      
pdaRelatedView color                   39  0.958621           0.884701   
               color_compare           33  0.725564           0.664415   
pdaView        color                   19  1.000000           1.000000   
               color_compare           32  0.941176           0.956805   

                              accuracy  sensitivity  specificity  
view           mode                                               
pdaRelatedView color          0.923077     1.000000     0.896552  
               color_compare  0.666667     0.571429     0.736842  
pdaView        color          1.000000     1.000000     1.000000  
               color_compare  0.875000     0.882353     0.866667

### Misses

In [14]:
true = df_results_clip_avg.target
pred_cls = (df_results_clip_avg.predicted>0.5).astype(int)

df_results_clip_avg[true != pred_cls]

,study,video,target,view,mode,predicted
9,study12,study12_dicom25,0.0,pdaRelatedView,color,0.543069
14,study16,study16_dicom27,0.0,pdaRelatedView,color_compare,0.674484
30,study20,study20_dicom115,1.0,pdaRelatedView,color_compare,0.375425
31,study20,study20_dicom119,1.0,pdaView,color_compare,0.458795
36,study20,study20_dicom88,1.0,pdaRelatedView,color_compare,0.252318
37,study20,study20_dicom94,1.0,pdaView,color_compare,0.380188
39,study20,study20_dicom98,1.0,pdaRelatedView,color_compare,0.362090
43,study22,study22_dicom13,0.0,pdaView,color_compare,0.544327
45,study23,study23_dicom64,0.0,pdaRelatedView,color,0.519349
55,study3,study3_dicom47,0.0,pdaRelatedView,color,0.622694


# Study-level results

In [15]:
df_results_study_avg = df_results_clip_avg.groupby(['study', 'target'], as_index=False).agg('mean')
display(df_results_study_avg)

compute_metrics(df_results_study_avg.target, df_results_study_avg.predicted)

,study,target,predicted
0,study1,1.0,0.883781
1,study11,0.0,0.256307
2,study12,0.0,0.445318
3,study13,0.0,0.221116
4,study14,0.0,0.279344
5,study16,0.0,0.492799
6,study17,0.0,0.119731
7,study17,1.0,0.791417
8,study2,1.0,0.827131
9,study20,0.0,0.201592


{'num_samples': 35,
 'roc_auc': 0.9482758620689655,
 'average_precision': 0.8999999999999999,
 'accuracy': 0.8571428571428571,
 'sensitivity': 0.8333333333333334,
 'specificity': 0.8620689655172413}